In [4]:
#### load packages, *.h5 folder and exp.h5 file location
%matplotlib widget

from essential_func import *
from analysis_data import *

qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

default_sample_dir = 'July-2021-Sample#7' #'July-sorted'  # '20-Dec' # 'July-2021-Sample#6'
csv_and_code_abs_directory = '/Users/bashit.a/Documents/Alzheimer/Codes/'
samples_csv = 'data_directory.csv'

dropdown_name_list, cwd, exp_folder = change_python_path(dropdown_name = default_sample_dir, csv_file_location = csv_and_code_abs_directory, samples_csv = samples_csv)

Python directory is set to load samples from : July-2021-Sample#7


In [35]:
### background subtraction
%matplotlib widget

file = '2029_Dentate_gyrus-roi0_masked.h5'
QSearchStart, QSearchEnd = 1.55, 1.8        # scaling regions 295,310 or may be 290,370
frame = 6430 #7048                          # amyloid frame    2428, 3101, 3923, 2190,  691, 1134, 458
tissue_frames = [6296,]                      # tissue frame, 
bkg_frame = [6000, 6001, 6002, 6003 ] # 973               # background frame
seek_mf = (-12,12,0.01)        #gray          # mf goes from -8 to +8 by 0.01
window_size = 5                             # window size for moving average filter
kernal_size = 7;
area_minQ = 1.0
area_maxQ = 1.45
mf_max = 2.5           #  mf must be less than this value
mf_min = 1.0           #  mf must be greater than this value

#	[6280, 6320, 6154]
### Tissue scaled background subtraction
data = Data_Analysis(file, qgrid2, directory=os.getcwd())
Iq   = data.bkg_sub(bkg_frame = bkg_frame)
mf, area, _ , _ = data.tissue_sub(frame, tissue_frames, scale_method = 'MSE', return_alg = 'one_tissue-fr' , \
                                                    area_minQ= area_minQ, area_maxQ = area_maxQ, mf_Qindices = (QSearchStart, QSearchEnd), \
                                                    seek_mf = seek_mf, mf_min=mf_min, mf_max=mf_max , \
                                                    window_size=window_size, show_result=False);
data.plot(plot_minQ=0.5, plot_maxQ=3)

### kernal sliding and getting background frames
Width, Height = width_height(file)
fr_idx = Snaking_frames_search(Width, Height)
tissue_frames = np.array(fr_idx.frame_idx_to_kernal_frames(kernal_size, frame))
print('kernal tissue frames - ',tissue_frames)

length = len(tissue_frames)
mf = {}; area = {'value':np.zeros(length)}
for i,tissue_frame in enumerate(tissue_frames):
    mf[i], area['value'][i],_,_ = data.tissue_sub(frame, tissue_frames, scale_method = 'MSE', return_alg = 'one_tissue-fr' , \
                                                    area_minQ= area_minQ, area_maxQ = area_maxQ, mf_Qindices = (QSearchStart, QSearchEnd), \
                                                    seek_mf = seek_mf, mf_min=mf_min, mf_max=mf_max , \
                                                    window_size=window_size, show_result=False);

print('descending order tissue frames')
tissue_frames_sorted = tissue_frames[np.argsort(area['value'],axis= None, kind='stable')[::-1] ] # descending order
print(tissue_frames_sorted)

/Users/bashit.a/miniforge3/envs/py38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/bashit.a/miniforge3/envs/py38/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: Invalid shape (1, 1043, 981) for image data

In [11]:
### Load all frames mf, and area data

sc_data = {}; sc_data['mf'] = np.zeros(data.n_patterns); sc_data['area'] = np.zeros(data.n_patterns)
for frame in range(data.n_patterns):
    sc_data['mf'][frame] , sc_data['area'][frame]   =  data.scaling_frame(frame, tissue_frames_sorted[0], mf_Qindices = (QSearchStart, QSearchEnd), seek_mf = seek_mf, window_size=window_size, show_result=False)

In [12]:
### Draw heat map depending on background subtracted descending area 
%matplotlib widget

file = file
sort_by = sc_data['area']

sorted_idx  = np.argsort(sort_by, kind='stable')[::-1][:50] # sc_data['area'].shape = (4941,)  # first 50 area sorted descending

diff_patterns = np.zeros(len(sort_by))
diff_patterns[sorted_idx] = sort_by[sorted_idx]
Width, Height = width_height(file)
img_orig = snaking(Width, Height, diff_patterns)

f,ax = plt.subplots()
plot_heat_map_from_data(img_orig, Width, Height, args = (f, ax), title= f'Heat Map {file}')
[print('Fr-',idx, 'value: ' , np.round(diff_patterns[idx],8)) for idx in sorted_idx]
pass

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fr- 2264 value:  0.00052844
Fr- 6548 value:  0.00049938
Fr- 677 value:  0.00049892
Fr- 6547 value:  0.0004633
Fr- 6556 value:  0.00042698
Fr- 6430 value:  0.0004251
Fr- 6546 value:  0.00040213
Fr- 2953 value:  0.00040124
Fr- 6555 value:  0.00038972
Fr- 6421 value:  0.0003875
Fr- 6672 value:  0.00038701
Fr- 1641 value:  0.0003768
Fr- 2002 value:  0.00037513
Fr- 6422 value:  0.00037433
Fr- 6429 value:  0.0003711
Fr- 1862 value:  0.0003566
Fr- 1759 value:  0.00035455
Fr- 6682 value:  0.00034807
Fr- 7309 value:  0.00034662
Fr- 6673 value:  0.00034253
Fr- 1760 value:  0.0003405
Fr- 3145 value:  0.00033697
Fr- 2381 value:  0.00033518
Fr- 7310 value:  0.00033406
Fr- 6431 value:  0.00033243
Fr- 5805 value:  0.00033181
Fr- 3646 value:  0.00032153
Fr- 6174 value:  0.00031985
Fr- 6173 value:  0.00031618
Fr- 6308 value:  0.00031284
Fr- 7556 value:  0.00031103
Fr- 493 value:  0.00030781
Fr- 122 value:  0.00030748
Fr- 4405 value:  0.00030705
Fr- 5534 value:  0.00030697
Fr- 2744 value:  0.00030659
Fr

In [5]:
### Produce heatmap and cluster frames by thresholding
%matplotlib widget

# spec
file = '2029_Dentate_gyrus-roi0_masked.h5'
scattering = '_WAXS2'
scattering_max = 0.0361
thr            = 0.034
scattering_inc = 0.001


# computation
Width, Height = width_height(file)
img_orig =  discritize_scattering(file, qgrid2, scattering, data_binning=True, bins=np.fromiter(drange(0, scattering_max, scattering_inc), float), directory=os.getcwd() )
gray_img,_ = global_thresholding(img_orig, thr, binary_inv = False)

# plot
f,axs = plt.subplots(nrows=1, ncols=2)
plot_heat_map_from_data(img_orig, Width, Height, args = (f, axs[0]), title= f'Heat Map {scattering}', cmap="jet")
plot_heat_map_from_data(gray_img, Width, Height, args = (f, axs[1]), title= f'Heat Map Thr at {thr}', cmap="jet")
plt.suptitle(f'{file}')
plt.tight_layout()

labeled_array, num_features = label(gray_img, np.ones((3,3)))
print('Total clusters = ', num_features)

Width, Height = labeled_array.shape[1], labeled_array.shape[0]
sna = snaking(Width, Height , np.arange(0, Width*Height))


dict_label = {}
for i in np.unique(labeled_array):
    if i!=0:
        dict_label[i] = sna[labeled_array==i]
        print(f'label {i}: {list(dict_label[i])}')

Minimum, Maximum Iq : 0.0192, 0.0473


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Total clusters =  40
label 1: [7807, 7690, 7680, 7681, 7565]
label 2: [7556, 7557, 7438, 7437, 7436, 7435, 7429, 7430, 7431, 7432, 7311, 7310, 7309, 7308, 7306]
label 3: [7048, 7049, 6937, 6936, 6935, 6922, 6923]
label 4: [7055, 6930]
label 5: [6798, 6799, 6800, 6684, 6683, 6682, 6681, 6671, 6672, 6673, 6674, 6558, 6557, 6556, 6555, 6546, 6547, 6548, 6431, 6430, 6429, 6420, 6421, 6422, 6304, 6303]
label 6: [6174, 6173, 6048, 6047]
label 7: [5931, 5930, 5912, 5913, 5806, 5805, 5785, 5680, 5679, 5659, 5660, 5554, 5553, 5552, 5533, 5534, 5535, 5428, 5427, 5426, 5408, 5409, 5301]
label 8: [5802]
label 9: [5058, 5057]
label 10: [4798, 4797, 4778]
label 11: [4677]
label 12: [4415, 4404]
label 13: [4278, 4279, 4280, 4163, 4162, 4161, 4152, 4153, 4154, 4037, 4036]
label 14: [4149, 4040]
label 15: [4156]
label 16: [3911, 3910, 3900, 3901, 3785, 3784, 3783, 3782]
label 17: [3749, 3750, 3685, 3684, 3683]
label 18: [3639, 3640, 3542]
label 19: [3287, 3286]
label 20: [3260]
label 21: [3271, 3154, 3

In [15]:
#### check generated heat maps collected frame data
%matplotlib widget
def exec_dropdown_value(frame):
    
    mf,area =  data.scaling_frame(frame, tissue_frame, mf_Qindices = (QSearchStart, QSearchEnd), seek_mf = seek_mf, window_size=window_size, show_result=True)
    data.plot()
    
frame = ipywidgets.Dropdown(options= sorted_area_idx,  description='Frame:',    disabled=False, continuous_update=False)
ipywidgets.interact(exec_dropdown_value, frame=frame)
print(sorted_area_idx)

NameError: name 'sorted_area_idx' is not defined

In [21]:
#### generate Radius of gyration
%matplotlib widget

from tqdm import tqdm

# specs
file = '2048_B8_masked.h5'
Nsplits  = 2                                        # number of lines to fit
StartIdx = 4                                        # including start index
LastIdx  = 52                                       # Excluding last index
frame    = 2223
max_frames = 40                                    # first 100 frame values
Rxc_str = ['Rxc-1', 'Rxc-2',]

# new part
data = Data_Analysis(file, qgrid2)
IqBS = data.bkg_sub(bkg_frame = 2223)

df_IqBS = pd.DataFrame(IqBS)    # convert to dataframe
list_max_frames = df_IqBS.sort_values(5,ascending=False)[:max_frames].index.to_list()   # 5-->th column values sorted in descending order and taking max 40 frames intensity to fit lines except are zero
print(list_max_frames)
Rxc = {}
n_patterns = len(IqBS)

for rxc_str in Rxc_str:
    Rxc[rxc_str] = np.zeros(n_patterns)     # except list_max_frames all values are zero

    for frame in tqdm(list_max_frames):
        df = optimize_best_lines(IqBS[frame], qgrid2, Nsplits, LastIdx, StartIdx,)   # get dataframe for the optimized one
        try:
            Rxc[rxc_str][frame] = df.iloc[df['rsq'].idxmax()][rxc_str]    
        except:
            print(f'Frame = {frame} {rxc_str} optimization failed - setting it to zero')
            Rxc[rxc_str][frame] = 0                 # set to zero in nan is found in try section
        
# plot heat maps
Width, Height = width_height(file)
f, axs = plt.subplots(1, len(Rxc_str), num=f'{file} Heat maps', figsize=(10,5))
for i, rxc_str in enumerate(Rxc_str):
    diff_patterns = Rxc[rxc_str]
    img_orig = snaking(Width, Height, diff_patterns)
    
    plot_heat_map_from_data(img_orig, Width, Height, args = (f, axs[i]), title= f'{file}- {rxc_str}')

  0%|          | 0/40 [00:00<?, ?it/s]/Users/bashit.a/Documents/Alzheimer/Codes/essential_func.py:832: RuntimeWarning: divide by zero encountered in log
  IqQ = np.log(IqBS*qgrid)                            # background subtracted Iq
  5%|▌         | 2/40 [00:00<00:02, 17.10it/s]

[2067, 1958, 1326, 2080, 1358, 2428, 2068, 4374, 2197, 2199, 4373, 1357, 2427, 2079, 1957, 2452, 2198, 1327, 4912, 2200, 1325, 4223, 2558, 4913, 4723, 2066, 4848, 4911, 1359, 4847, 2437, 2081, 2323, 4222, 2322, 2196, 1959, 2320, 2438, 2453]


 12%|█▎        | 5/40 [00:00<00:01, 19.54it/s]/Users/bashit.a/Documents/Alzheimer/Codes/essential_func.py:832: RuntimeWarning: invalid value encountered in log
  IqQ = np.log(IqBS*qgrid)                            # background subtracted Iq
/Users/bashit.a/Documents/Alzheimer/Codes/essential_func.py:859: RuntimeWarning: invalid value encountered in sqrt
  df.loc[idx,f'Rxc-{i+1}'] = np.round(np.sqrt(-df.loc[idx,f'slope-{i+1}']*2),4)
100%|██████████| 40/40 [00:01<00:00, 22.18it/s]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
### get individual frame fitting of radius of gyration
%matplotlib widget
# specs
frame = 2067
# verification
df = optimize_best_lines(IqBS[frame], qgrid2, Nsplits, LastIdx, StartIdx, print_summary=True, show_plot=True)
print(df)

Total Combinations = 45

Summary of results Rsq -- 
 0               4.0000
1              18.0000
2              18.0000
3              52.0000
Tpoints-1      14.0000
qgridL-1        0.0090
qgridH-1        0.0230
slope-1     -4833.9374
Rxc-1          98.3254
I(0)-1          0.9474
rsq-1           0.9472
std_err-1     329.6247
Tpoints-2      34.0000
qgridL-2        0.0230
qgridH-2        0.0640
slope-2      -698.0548
Rxc-2          37.3646
I(0)-2         -1.2604
rsq-2           0.9421
std_err-2      30.5975
rsq             1.8893
Name: 12, dtype: float64
More Summary of results Rsq -- 
     0   1   2   3  Tpoints-1  qgridL-1  qgridH-1     slope-1     Rxc-1  \
0   4   6   6  52          2     0.009     0.011 -12982.1460  161.1344   
1   4   7   7  52          3     0.009     0.012 -11530.7305  151.8600   
2   4   8   8  52          4     0.009     0.013 -10360.3461  143.9468   
3   4   9   9  52          5     0.009     0.014  -9429.4692  137.3279   
4   4  10  10  52          6     0.0

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

    0   1   2   3  Tpoints-1  qgridL-1  qgridH-1     slope-1     Rxc-1  \
0   4   6   6  52          2     0.009     0.011 -12982.1460  161.1344   
1   4   7   7  52          3     0.009     0.012 -11530.7305  151.8600   
2   4   8   8  52          4     0.009     0.013 -10360.3461  143.9468   
3   4   9   9  52          5     0.009     0.014  -9429.4692  137.3279   
4   4  10  10  52          6     0.009     0.015  -8636.6298  131.4278   
5   4  11  11  52          7     0.009     0.016  -7878.8447  125.5296   
6   4  12  12  52          8     0.009     0.017  -7300.9062  120.8380   
7   4  13  13  52          9     0.009     0.018  -6807.7529  116.6855   
8   4  14  14  52         10     0.009     0.019  -6297.4656  112.2271   
9   4  15  15  52         11     0.009     0.020  -5884.7404  108.4872   
10  4  16  16  52         12     0.009     0.021  -5497.3718  104.8558   
11  4  17  17  52         13     0.009     0.022  -5129.8891  101.2906   
12  4  18  18  52         14     0.009

In [15]:
pd.set_option('display.max_columns',None)
df.style.background_gradient()
df[df['rsq'] > 1.86]                      # Nsplits
#df[df['rsq-3'] > 0.99]['Tpoints-3'].idxmax()      # Nsplits
#plot_linear_lines(xData, yData, best_indices)

,0,1,2,3,Tpoints-1,qgridL-1,qgridH-1,slope-1,Rxc-1,I(0)-1,rsq-1,std_err-1,Tpoints-2,qgridL-2,qgridH-2,slope-2,Rxc-2,I(0)-2,rsq-2,std_err-2,rsq
2,4,8,8,52,4,0.009,0.013,-10360.3461,143.9468,1.6805,0.9908,707.4576,44,0.013,0.064,-921.5366,42.9310,-0.7508,0.8726,54.3384,1.8634
3,4,9,9,52,5,0.009,0.014,-9429.4692,137.3279,1.5856,0.9869,626.9592,43,0.014,0.064,-893.2780,42.2677,-0.8131,0.8823,50.9652,1.8692
4,4,10,10,52,6,0.009,0.015,-8636.6298,131.4278,1.4986,0.9828,570.7240,42,0.015,0.064,-866.6187,41.6322,-0.8723,0.8912,47.8814,1.8740
5,4,11,11,52,7,0.009,0.016,-7878.8447,125.5296,1.4093,0.9761,551.1112,41,0.016,0.064,-840.9667,41.0114,-0.9297,0.9000,44.8855,1.8761
6,4,12,12,52,8,0.009,0.017,-7300.9062,120.8380,1.3361,0.9733,493.7115,40,0.017,0.064,-817.0807,40.4248,-0.9836,0.9078,42.2534,1.8811
7,4,13,13,52,9,0.009,0.018,-6807.7529,116.6855,1.2693,0.9709,445.6174,39,0.018,0.064,-794.8459,39.8709,-1.0342,0.9144,39.9717,1.8853
8,4,14,14,52,10,0.009,0.019,-6297.4656,112.2271,1.1953,0.9642,429.2031,38,0.019,0.064,-772.9985,39.3192,-1.0843,0.9214,37.6173,1.8856
9,4,15,15,52,11,0.009,0.020,-5884.7404,108.4872,1.1313,0.9607,396.8797,37,0.020,0.064,-752.8977,38.8046,-1.1309,0.9272,35.6586,1.8879
10,4,16,16,52,12,0.009,0.021,-5497.3718,104.8558,1.0673,0.9559,373.3956,36,0.021,0.064,-733.6226,38.3046,-1.1760,0.9327,33.8078,1.8886
11,4,17,17,52,13,0.009,0.022,-5129.8891,101.2906,1.0027,0.9497,355.9882,35,0.022,0.064,-714.7376,37.8084,-1.2206,0.9383,31.9132,1.8880


In [16]:
%matplotlib widget

# specs
frame = 617    # 2048_B8_masked frame = 617 Rxc_2 = nan
QQ  = np.square(qgrid2)                            # squaring q values
IqQ = np.log(IqBS[frame]*qgrid2)                         # background subtracted Iq

# semi specs
xData = QQ
yData = IqQ

# computation
def update_plot(Q1, Q2H):
    Q1L, Q1H = Q1
    plot_linear_lines(xData=xData, yData=yData, indices=[Q1L,Q1H,Q1H,Q2H])

Q1 =  ipywidgets.IntRangeSlider(min=0, max=109, value = [2, 50], step=1)
Q2H = ipywidgets.IntSlider(min=0, max=109, value = 109, step=1)

ipywidgets.interact(update_plot, Q1=Q1, Q2H=Q2H)

In [50]:
### crysol plots
%matplotlib widget

df1 = pd.read_csv("/Users/bashit.a/Documents/Alzheimer/PDB/5OQV/5kpoints/5oqv3deg5k.csv")
df2 = pd.read_csv("/Users/bashit.a/Documents/Alzheimer/PDB/5OQV/5oqv3deg.csv")
df3 = pd.read_csv("/Users/bashit.a/Documents/Alzheimer/PDB/5OQV/5OQV-3deg-Deepti.csv")

plt.plot(df1['q'], np.log(df1['Iq']),label='5kpoints');
plt.plot(df2['q'], np.log(df2['Iq']), label='5OQV');
plt.plot(df3['q'], np.log(df3['Iq']), label='5OQV-D');
plt.xlabel('q');
plt.ylabel('log(Iq)');
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
%matplotlib widget
idx_l, idx_u, valid_diff_values = valid_idx_search(np.array(df1['q']), np.array([df1['Iq']]), show_q = True)
Iq = np.interp(qgrid2, np.array(df1['q']), np.array(df1['Iq']))
idx_l, idx_u, valid_diff_values = valid_idx_search(qgrid2, Iq.reshape(1,-1))

plt.plot(qgrid2, np.log(Iq))

In [4]:
### plot PDB twisted files

%matplotlib widget
import pandas as pd
import pylab as plt
file = '/Users/bashit.a/Documents/Alzheimer/PDB/CSV/6hrf.csv';
df = pd.read_csv(file);
plt.plot(df['q'], np.log(df['0']),label='0');
plt.plot(df['q'], np.log(df['1']),label='1');
plt.plot(df['q'], np.log(df['2']),label='2');
plt.plot(df['q'], np.log(df['3']),label='3');
plt.plot(df['q'], np.log(df['-1']),label='-1');
plt.plot(df['q'], np.log(df['-2']),label='-2');
plt.plot(df['q'], np.log(df['-3']),label='-3');
plt.title(file)
plt.xlabel('q')
plt.ylabel('log(I)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
%matplotlib widget
import mplcursors
from matplotlib.widgets import CheckButtons

from essential_func import *
from ipywidgets import Layout, Button, Box, BoundedIntText, IntText, FloatText, Textarea, Dropdown, Label, FloatSlider, IntSlider, Checkbox

### updating list
def get_files_list(dropdown_name , seek_strs = ['.h5','.csv', '.xlsx']):
    df  = pd.read_csv(os.path.join(csv_and_code_abs_directory,samples_csv))
    idx = df[df["dropdown-name"]==dropdown_name].index
    directory = df['bnl-scan-sample-dir'][idx].values[0]
    files_list = []
    for seek_str in seek_strs:
#         print(seek_strs)
        files_list.append(cwd_files_search_with(seek_str, directory = directory, ))
#     print(flatten(files_list))
    return directory, flatten(files_list)

### general layout
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)
### main menu widgets
bkg_sub   = Checkbox(value = False)
normalize = Dropdown(options=['min-max', 'No'], value = 'min-max')
n_filt    = BoundedIntText(value =4, min=1, max=20, style = {'description_width': 'initial'})
y_axis    = Dropdown(options=['Iq', 'log(Iq)', 'qlog(Iq)'])
x_axis    = Dropdown(options=['q', 'q^2'])
fr_cat    = BoundedIntText(value =1, min=1, max=10, style = {'description_width': 'initial'}, description = 'No. of Frames Category ' , layout=form_item_layout)

main_menu = Box(
    [
    Box([Label(value='Background Subtraction '),bkg_sub], layout=form_item_layout),
    Box([Label(value='Normalize '),  normalize], layout=form_item_layout), 
    Box([Label(value='N-point Filt. '), n_filt], layout=form_item_layout), 
    Box([Label(value='Y-axis. '),       y_axis], layout=form_item_layout), 
    Box([Label(value='X-axis. '),       x_axis], layout=form_item_layout),
    ], 
    layout= Layout(
        display='flex',
        flex_flow='column',
        border='solid 2px',
        align_items='stretch',
        width='30%')
    )
### create dynamic function name
def bindFunction(name):
    def func_(*args):
        file[name].options = get_files_list(dropdown_name = directory[name].value)[1]
    func_.__name__ = 'on_value_change_'+str(name)
    return func_

def create_form_items(total_categories, bkg_subtraction):
    ## form widgets
    global directory, file, bkg_fr, frames, form_items, idx_
    directory = {}; file = {}; bkg_fr={}; frames={}; form_items = []
    for i in range(total_categories):
        directory[i] = Dropdown(options = dropdown_name_list)
        file[i]      = Dropdown(options = get_files_list(dropdown_name = dropdown_name_list[0] , seek_strs = ['.h5','.csv', '.xlsx'])[1])
        bkg_fr[i]    = IntText() if bkg_subtraction else IntText(value = 55555, disabled=True)   ### default back_sub frame means bkg_sub is false
        frames[i]    = Textarea()
        
        directory[i].observe(bindFunction(i), names='value')
        
        form_items.append([
                Box([Label(value='Directory'),        directory[i]],  layout=form_item_layout),
                Box([Label(value='File'),             file[i]],       layout=form_item_layout),
                Box([Label(value='Background Frame'), bkg_fr[i]],     layout=form_item_layout),
                Box([Label(value='Frames'),           frames[i]],     layout=form_item_layout)
                ]) #* total_categories

    form = Box(flatten(form_items),
    layout=Layout(
        display='flex',
        flex_flow='column',
        border='solid 1px',
        align_items='stretch',
        width='35%')
    )
    display(main_menu, form)

    
def on_button_clicked(*args):
#     if bkg_sub.value == False:
#         saxs_plots, waxs_plots = get_plots(directories = ['/Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sample-6/', \
#                                                          '/Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sample-1/'], \
#                                            files=['2029_Dentate_gyrus-roi0_masked.h5', '2040_FC-roi0.h5'], \
#                                            qgrid=qgrid2, normalize = True, \
#                                            frames = [[6558,981],[150,120]],  x_axis='q' , y_axis='Iq', bkg_fr = [1480, 100 ])
    
    global directories_arg, files_arg, frames_arg, bkg_fr_arg
    directories_arg = []; files_arg=[]; frames_arg = []; bkg_fr_arg = [];

    for idx in range(fr_cat.value):
        directories_arg.append(get_files_list(dropdown_name = directory[idx].value)[0])
        files_arg.append(file[idx].value)
        frames_arg.append([int(i) for i in frames[idx].value.split(",") if i!=''])
        bkg_fr_arg.append(bkg_fr[idx].value)
    print(directories_arg, files_arg, frames_arg, bkg_fr_arg)
                               
button     = ipywidgets.Button(description='Go!');                                      
button.style.button_color = "green";
button.on_click(on_button_clicked)

out = ipywidgets.interactive_output(create_form_items, {'total_categories' : fr_cat, 'bkg_subtraction' : bkg_sub})
display(fr_cat, out, button)

BoundedIntText(value=1, description='No. of Frames Category ', layout=Layout(display='flex', flex_flow='row', …

Output()

Button(description='Go!', style=ButtonStyle(button_color='green'))

In [11]:
# 1-d plot for high intensities
%matplotlib widget
import mplcursors
from matplotlib.widgets import CheckButtons

# plot
f, ax = plt.subplots(2,2, figsize = (16,8), num=f'data', constrained_layout = False)
plt.subplots_adjust(left=0.15)

def saxs_waxs_plot_data_from_Iq(xData, yData, waxs_idx_start, waxs_idx_end, saxs_idx_start, saxs_idx_end, n_filt, normalize, y_axis):

    if y_axis == 'Iq':
        yData = yData;                          ax[0,0].set_ylabel("$I$");                 ax[1,0].set_ylabel("$I$");
    elif y_axis == 'log(Iq)':
        yData = np.log(yData);             ax[0,0].set_ylabel("$log(I)$");            ax[1,0].set_ylabel("$log(I)$");
    elif y_axis == 'qlog(Iq)':
        yData = np.log(yData*qgrid);      ax[0,0].set_ylabel("$qlog(I)$");           ax[1,0].set_ylabel("$qlog(I)$");
    else:
        print('No yData')

    # filter output using convolution
    ### filter coefficients
    b = np.ones((n_filt))/n_filt    # numerator co-effs of filter transfer function
    a = np.ones(1)                  # denominator co-effs of filter transfer function
    yData = np.convolve(b, yData, mode='same') if n_filt>1 else yData         

    # extract saxs and waxs data
    yData_waxs = yData[waxs_idx_start : waxs_idx_end]
    yData_saxs = yData[saxs_idx_start : saxs_idx_end]

    # normalize saxs and waxs data
    yData_waxs = yData_waxs/np.nanmax(yData_waxs) if normalize=='min-max' else yData_waxs   # normalized by maximum value
    yData_saxs = yData_saxs/np.nanmax(yData_saxs) if normalize=='min-max' else yData_saxs   # normalized by maximum value

    return yData_saxs, yData_waxs

def get_plots(directories, files, qgrid, frames,  x_axis , y_axis, normalize='min-max', n_filt=1, bkg_fr = False,  saxs_scale = 'linear', waxs_scale = 'linear'):
    
    ### specs
    waxs_idx_start     = 130     # qgrid2[130] = 0.4
    waxs_idx_end       = 450     # qgrid2[450] = 2.0
    saxs_idx_start     = 5       # qgrid2[2]   = 0.007
    saxs_idx_end       = 50      # qgrid2[2]   = 0.06
    
    ### create empty lists to return
    saxs_plots = []    
    waxs_plots = []

    xData = qgrid if x_axis == 'q' else np.square(qgrid)

    bkg_fr = [[] for _ in range(len(files))] if bkg_fr==False else bkg_fr

    for idx, (dir_, file_, frame_, bkg_fr_) in enumerate(zip(directories, files, frames, bkg_fr)):    

        try:

            if file_.endswith('csv') or file_.endswith('xlsx'):
                df = pd.read_csv(dir_+file_)
                Iq = np.interp(qgrid, np.array(df['q']), np.array(df['Iq']))   # inpterpolation

                yData_saxs, yData_waxs = saxs_waxs_plot_data_from_Iq(xData, yData = Iq, waxs_idx_start=waxs_idx_start, waxs_idx_end=waxs_idx_end, saxs_idx_start=saxs_idx_start, saxs_idx_end=saxs_idx_end, \
                                                                     n_filt=n_filt, normalize=normalize, y_axis=y_axis,)

                waxs_plots.append(ax[0,0].plot(xData[waxs_idx_start:waxs_idx_end],  yData_waxs, visible=False, label = file_.split(".")[0]))   # here dataset has only one frame
                saxs_plots.append(ax[1,0].plot(xData[saxs_idx_start:saxs_idx_end] , yData_saxs, visible=False, label = file_.split(".")[0]))

            else:
                print('reading file ... ', dir_, file_, frame_, bkg_fr_)
                data = Data_Analysis(file_ , qgrid = qgrid, window_size=1, directory = dir_)

                Iq = data.bkg_sub(bkg_frame = bkg_fr_) if bkg_fr_!=55555 else data.Iq

                for frame in frame_:
                    yData_saxs, yData_waxs = saxs_waxs_plot_data_from_Iq(xData, yData = Iq[frame], waxs_idx_start=waxs_idx_start, waxs_idx_end=waxs_idx_end, saxs_idx_start=saxs_idx_start, saxs_idx_end=saxs_idx_end, \
                                                                     n_filt=n_filt, normalize=normalize, y_axis=y_axis,)

                    waxs_plots.append(ax[0,0].plot(xData[waxs_idx_start:waxs_idx_end],  yData_waxs, visible=True, label = f'{idx}#{frame}'))   # here dataset has only one frame
                    saxs_plots.append(ax[1,0].plot(xData[saxs_idx_start:saxs_idx_end] , yData_saxs, visible=True, label = f'{idx}#{frame}'))
        except:
            print('file reading failed : ', {file_})

    ax[0,0].set(xlabel = "$q (\AA^{-1})$" if x_axis == 'q' else "$q^2 (\AA^{-1})$" , xscale = 'linear', yscale = waxs_scale)
    ax[1,0].set(xlabel = "$q (\AA^{-1})$" if x_axis == 'q' else "$q^2 (\AA^{-1})$" , xscale = 'linear', yscale = saxs_scale)

    return saxs_plots, waxs_plots

### create SAXS WAXS Plot
saxs_plots, waxs_plots = get_plots(directories = directories_arg, files=files_arg, qgrid=qgrid2, normalize=normalize.value, n_filt=n_filt.value, frames = frames_arg,  x_axis=x_axis.value , y_axis=y_axis.value, bkg_fr = bkg_fr_arg)
# plot heatmap
#plot_heat_map_from_data(img_orig, Width, Height, args = (f, ax[0,1]), cmap='jet')
#plt.tight_layout()

# Make checkbuttons with all plotted lines with correct visibility
rax = plt.axes([0.005, 0.1, 0.05, 0.6])
labels = [str(line[0].get_label()) for line in saxs_plots]   #label is a string
visibility = [line[0].get_visible() for line in saxs_plots]
check = CheckButtons(rax, labels, visibility, )
for r,l in zip(check.rectangles, check.lines):
    r.set_width(0.1)
    r.set_alpha(0.4)
    for ll in l:
        ll.set_linewidth(8)

# [ll.set_linewidth(8) for l in check.lines for ll in l]
def func(label):
    index = labels.index(label)
    saxs_plots[index][0].set_visible(not saxs_plots[index][0].get_visible())
    waxs_plots[index][0].set_visible(not waxs_plots[index][0].get_visible())

    ax[0,0].legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize='xx-small')
    ax[1,0].legend(loc='upper right', bbox_to_anchor=(1, 1), fontsize='xx-small')

    plt.draw()

check.on_clicked(func)
plt.subplot_tool()
#mplcursors.cursor()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

reading file ...  /Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sample-7/ 2109_EC-roi1_masked.h5 [250, 249, 248, 247, 246, 253, 254, 255, 256, 257, 376, 375, 374, 373, 372, 379, 380, 381, 382, 383, 502, 501, 500, 499, 498] 3469


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
## Do cluster labeling
import cv2
from scipy.ndimage import label

img_gray = np.asarray(img_orig, dtype=np.uint8)
_, thresh_img = cv2.threshold(img_gray, 0, 255,cv2.THRESH_BINARY); #plt.imshow(thresh_img, cmap='gray')
labeled_array, num_features = label(thresh_img, np.ones((3,3)))
print('Total clusters = ', num_features)

Width, Height = labeled_array.shape[1], labeled_array.shape[0]
sna = snaking(Width, Height , np.arange(0, Width*Height))

[print('label = ', i, sna[labeled_array==i]) for i in np.unique(labeled_array) if i!=0]

Total clusters =  0


[]

In [21]:
from generate_files import *
save_mat('IqBS', ('IqBS', 'qgrid2'), (IqBS, qgrid2))

In [8]:
### Correlation coefficients analysis

Width, Height = 61, 81
sna = snaking(Width, Height , np.arange(0, Width*Height))
print(sna)

grid = np.zeros((Height,Width),dtype=object)
for i in range(0,Height):
    for j in range(0,Width):
        grid[i,j]=(i,j)
grid = grid[::-1]
print(grid)

[[4880 4881 4882 ... 4938 4939 4940]
 [4879 4878 4877 ... 4821 4820 4819]
 [4758 4759 4760 ... 4816 4817 4818]
 ...
 [ 122  123  124 ...  180  181  182]
 [ 121  120  119 ...   63   62   61]
 [   0    1    2 ...   58   59   60]]
[[(80, 0) (80, 1) (80, 2) ... (80, 58) (80, 59) (80, 60)]
 [(79, 0) (79, 1) (79, 2) ... (79, 58) (79, 59) (79, 60)]
 [(78, 0) (78, 1) (78, 2) ... (78, 58) (78, 59) (78, 60)]
 ...
 [(2, 0) (2, 1) (2, 2) ... (2, 58) (2, 59) (2, 60)]
 [(1, 0) (1, 1) (1, 2) ... (1, 58) (1, 59) (1, 60)]
 [(0, 0) (0, 1) (0, 2) ... (0, 58) (0, 59) (0, 60)]]


In [15]:
frame = 4940
def frame_to_idx(frame):
    idx = [i for i in grid[sna==frame]]
    return idx

def idx_to_frame(l):
    idx = []
    for i in l:
        a, b = i
        x = [i==(a,b) for i in grid[Height-a-1]]
        idx.append(sna[Height-a-1, np.where(x)].tolist())
        print(idx)
    return flatten(idx)

l = frame_to_idx(frame)
print(frame_to_idx(frame))
print(idx_to_frame(l))

[(80, 60)]
[[[4940]]]
[4940]


In [32]:
### locating valid neighbor indices 3*3 kernel
a, b = 80, 60
l = []
for i in range(a-1,a+2):
    for j in range(b-1,b+2):
        if (i,j) !=(a,b) and i>=0 and j>=0 and i<Height and j <Width:
            l.append((i,j))
print(l)

[(79, 59), (79, 60), (80, 59)]


In [33]:
## look for frames gotten from previous indices
idx = []
for i in l:
    a, b = i
    x = [i==(a,b) for i in grid[Height-a-1]]
    idx.append(sna[Height-a-1, np.where(x)].tolist())
flatten(idx)

[4820, 4819, 4939]

In [26]:
from scipy.io import loadmat
annots = loadmat('2048_B8_Iq.mat')

In [10]:
### Correlation coefficient calculation on individual frame
%matplotlib widget
from functools import reduce

def coeffs_map(file, ref_frame, thr_coeff, search_q, normalize, L):
    b = np.ones((1,L))/L    # numerator co-effs of filter transfer function
    a = np.ones(1)          # denominator co-effs of filter transfer function

    q_min, q_max = search_q
    Width, Height = width_height(file)
    Iq = read_Iq(file, 'merged')

    Iq = signal.fftconvolve(Iq,b,mode='same',) if L>1 else Iq         # filter output using convolution
    Iq = Iq/np.max(Iq,axis=1).reshape(-1,1) if normalize else Iq      # test how normalizing affecing data points print(Iq[0][:10])

    q_min_idx, q_max_idx = qgrid_to_indices(qgrid2, q_min), qgrid_to_indices(qgrid2, q_max)
    sna = snaking(Width, Height)
    global cluster
    cluster = []; coeffs = []
    for frame in range(Width * Height):
        coeff = np.corrcoef(Iq[ref_frame][q_min_idx:q_max_idx], Iq[frame][q_min_idx:q_max_idx])[0,1]
        if coeff >= thr_coeff:
            cluster.append(frame)
            coeffs.append(coeff)
    A = np.array([np.zeros((Height,Width)),sna])   # zero values matrix (A[0]=0) with frame numbers depth (A[1]=frames)

    A = from_clusterFr_ceffs_to_matrix(A, cluster, coeffs)
    
    f,ax = plt.subplots()
    plot_heat_map_from_data(A[0], Width, Height, args = (f,ax), title= f'Frame {ref_frame} coeffs map', cmap='jet')  # inferno

#coeffs_map(file, ref_frame, thr_coeff,  search_q = (q_min, q_max))

files = cwd_files_search_with('.h5')
dropdown = ipywidgets.Dropdown(options= files, value = files[0], description='File : ', disabled=False)
frame = ipywidgets.IntSlider(value=0,                   min=0,           max= 15000, step=1, description='Frame : ', continuous_update=False, layout=ipywidgets.Layout(width='50%')) # reduce(lambda x,y: x*y-1 , width_height(dropdown.value) )
thr_coeff = ipywidgets.FloatSlider(value = 0.96 ,       min= 0,       max=1,    step=0.001 , description='THR Coeff:', continuous_update=False, layout=ipywidgets.Layout(width='50%'), readout=True, readout_format='.3f')    
search_q = ipywidgets.FloatRangeSlider( value=(1, 2), min=qgrid2[0],   max=qgrid2[-1], step=0.05,   description='q Range:',  disabled=False,  continuous_update=False,  orientation='horizontal',  readout=True, readout_format='.3f',layout=ipywidgets.Layout(width='50%') )
normalize = ipywidgets.Checkbox(value=True, description='Normalize', disabled=False, indent=False )
n_point = ipywidgets.IntSlider(value=4,                   min=1,         max=10, step=1, description='N-point : ', continuous_update=False, layout=ipywidgets.Layout(width='30%'))


ipywidgets.interactive(coeffs_map, file=dropdown, ref_frame=frame, thr_coeff=thr_coeff, search_q= search_q, normalize= normalize, L = n_point)

interactive(children=(Dropdown(description='File : ', options=('2029_Dentate_gyrus-roi0.h5', '2029_Dentate_gyr…

In [56]:
### checking convolution funcitons
L = 5
b = np.ones((1,L))/L
x = np.array([1,2,3,5,2,7]).reshape(1,-1)
print(signal.convolve(x,b, mode='same'))

L=5                        # L-point filter
b = (np.ones((1,L)))/L     # numerator co-effs of filter transfer function
a = np.ones(1)             # denominator co-effs of filter transfer function
y = signal.fftconvolve(x,b,mode='same',)   # filter output using convolution
print(y)
qgrid2[375]

[[1.2 2.2 2.6 3.8 3.4 2.8]]
[[1.2 2.2 2.6 3.8 3.4 2.8]]


1.6250000000000013

In [26]:
#### Mean shift unsupervised clustering algorithm ~ 3 mins to execute
from sklearn.cluster import MeanShift, estimate_bandwidth

# The following bandwidth can be automatically detected using
ms = MeanShift(n_jobs=-1)                 # sklearn mean-shift n_jobs = -1 means using all processors
ms.fit(X)                        # train mean shift algorithm
labels = ms.labels_              # get labels of trained data
plot_labels(file, labels)

number of estimated clusters : 15


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
%matplotlib widget

### import package
from sklearn.manifold import TSNE, MDS, LocallyLinearEmbedding
from functools import partial

### GMM model
def train_GMM(n_components):
    from sklearn.mixture import GaussianMixture               # import package
    gaussian_model = GaussianMixture(n_components=n_components)          # define the model
    gaussian_model.fit(X)                                     # train the model
    labels = gaussian_model.predict(X)                        # assign each data point to a cluster
    return labels

### t-SNE
def plot_tsne(X, labels, perplexity, n_iter):
    X_embedded = TSNE(n_components=3, perplexity=perplexity, n_iter=n_iter, init='pca', learning_rate=200, random_state=0, n_jobs=-1).fit_transform(X)
    #X_embedded = MDS(n_components=3, max_iter=100, n_init=1).fit_transform(X)
    
    # Set-up manifold methods
#     LLE = partial(LocallyLinearEmbedding, n_neighbors=10, n_components=3, eigen_solver='auto')
#     X_embedded = LLE(method='modified').fit_transform(X)
    
    tsne_data = np.vstack((X_embedded.T, labels)).T    # creating a new data frame which help us in ploting the result data
    #tsne_df = pd.DataFrame (data=tsne_data, columns= ("Dim_1", "Dim_2", "Dim_3", "label"))
    #sns.FacetGrid(tsne_df, hue="label", size=6).map(plt.scatter, 'Dim_1', 'Dim_2', 'Dim_3', 'label')
    fig = plt.figure()
    ax  = fig.add_subplot(projection='3d')
    ax.scatter(*zip(*tsne_data[:,:3]), c=tsne_data[:,3])
    plt.show()

labels = train_GMM(n_components=2)
plot_labels(file, labels, title='gaussian')
plot_tsne(X, labels, perplexity=50, n_iter=1000)

NameError: name 'X' is not defined

In [4]:
#### GMM and t-SNE plot
import functools

### figures output widgets
out1 = ipywidgets.Output()
out2 = ipywidgets.Output()

### GMM model
def train_GMM(X, n_components):
    from sklearn.mixture import GaussianMixture               # import package
    gaussian_model = GaussianMixture(n_components=n_components)          # define the model
    
    from sklearn import cluster
    #birch = cluster.Birch(n_clusters=n_components)
    
    gaussian_model.fit(X)                                     # train the model
    labels = gaussian_model.predict(X)                        # assign each data point to a cluster
    return labels

@out1.capture(clear_output=True)
def unsupervised_map(file, labels, clusters):
    f1, ax1 = plt.subplots(num=1)
    plot_labels(file, labels, title='GMM', args = (f1, ax1), cmap = discrete_cmap( N= len(np.unique(labels)), base_cmap = 'brg') )   # discrete_cmap( N= len(np.unique(labels)), base_cmap = 'brg') 'Pastel1'

@out2.capture(clear_output=True)
def manifold_plot(X, labels, perplexity, n_iter, animation):
    tsne_data = manifold_tsne(X, perplexity, n_iter, dim=3)
    print(animation)
    f2, ax2 = plt.subplots(num=2)
    plot_3d(tsne_data, labels, args = (f2,ax2), cmap = discrete_cmap( N= len(np.unique(labels)), base_cmap = 'brg'))  # discrete_cmap( N= len(np.unique(labels)), base_cmap = 'bwr')   'Pastel1'
    if animation: plot_3d_animation(tsne_data, labels, elev=35, azim=60, anim_frames=100, anim_interval=10)

def on_button_clicked_params():
    return dropdown.value, search_q.value, n_point.value, normalize.value, clusters.value, perplexity.value, n_iter.value, animation.value

def on_button_clicked(_):   # 
    %matplotlib widget
    
    ### get function arguments
    dropdown, search_q, n_point, normalize, clusters, perplexity, n_iter, animation = on_button_clicked_params()
    
    ### Data preprocessing
    q_min, q_max = search_q
    X = file_preprocess(file = dropdown, L=n_point, qgrid=qgrid2, q_min=q_min, q_max=q_max, normalize=normalize)
    
    ### Unsupervised model training and visualizing
    labels = train_GMM(X, n_components=clusters)
    labels = sort_labels(labels)
    unsupervised_map(dropdown, labels, clusters)
    manifold_plot(X, labels, perplexity, n_iter, animation)
    
### file selection and operation widgets  
files      = cwd_files_search_with('.h5')
dropdown   = ipywidgets.Dropdown(options= files, value = files[0], description='File : ', disabled=False)
search_q   = ipywidgets.FloatRangeSlider( value=(1.0, 2.0), min=qgrid2[0],   max=qgrid2[-1], step=0.0001,   description='q Range:',  disabled=False,  continuous_update=False,  layout=ipywidgets.Layout(width='40%'), orientation='horizontal',  readout=True, readout_format='.3f',)
n_point    = ipywidgets.IntSlider(value=4,                   min=1,         max=10, step=1, description='N-point : ', continuous_update=False, )
normalize  = ipywidgets.Checkbox(value=False, description='Normalize', disabled=False, indent=False )

### clustering and manifold widgets
clusters   = ipywidgets.IntSlider(value=4, min=1, max=20, step=1, description='clusters', continuous_update=False          )
perplexity = ipywidgets.IntSlider(value=30, min=10, max=1000, step=10, description='perplexity', continuous_update=False   )
n_iter     = ipywidgets.IntSlider(value=1000, min=500, max=10000, step=500, description='n_iter', continuous_update=False  )
button     = ipywidgets.Button(description='Go!');                                      button.style.button_color = "green";

button.on_click(on_button_clicked)

### animation object
animation  = ipywidgets.Checkbox(value=False, description='Animation', disabled=False, indent=False )


display(ipywidgets.HBox([dropdown, search_q  ]))
display(ipywidgets.HBox([n_point, normalize, animation ]))
display(ipywidgets.HBox([ipywidgets.VBox([clusters, perplexity, n_iter, button]), out1, out2]))

In [8]:
import functools
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Click Me!")
display(button)

def fun(a):
    print(a)

def on_button_clicked(b, rs_="some_default_string"):
    fun(rs_)

button.on_click(functools.partial(on_button_clicked, rs_="abcdefg"))

Button(description='Click Me!', style=ButtonStyle())

In [2]:
### difference intensity heatmap
out = ipywidgets.Output()

@out.capture(clear_output=True)
def on_button_clicked(_):   
    %matplotlib widget

    Width, Height = width_height(file=dropdown.value)
    
    ### Data preprocessing
    q_min, q_max = search_q.value
    X = file_preprocess(file = dropdown.value, L=n_point.value, qgrid=qgrid2, q_min=q_min, q_max=q_max, normalize=False)
    diff_patterns = find_rep_value(qgrid2, X, method = 'circ')
    filtered_diff_patterns = snaking(Width, Height, diff_patterns)
    filtered_diff_patterns = filtered_diff_patterns.reshape(Height,Width)

    ### original data
    Iq = read_Iq(file = dropdown.value, scattering = 'merged')
    q_min_idx, q_max_idx = qgrid_to_indices(qgrid2, q_min), qgrid_to_indices(qgrid2, q_max)
    Iq = Iq[:,q_min_idx:q_max_idx]
    diff_patterns = find_rep_value(qgrid2, Iq, method = 'circ')
    img_orig = snaking(Width, Height, diff_patterns)
    img_orig = img_orig.reshape(Height,Width)

    subtracted = (img_orig-filtered_diff_patterns).reshape(Height,Width)

    fig, [ax1, ax2, ax3] = plt.subplots(nrows=1, ncols=3, figsize=(16,3))
    
    plot_heat_map_from_data(img_orig, Width, Height, args = (fig, ax1), title= 'original', cmap="viridis")
    plot_heat_map_from_data(filtered_diff_patterns, Width, Height, args = (fig, ax2), title= 'filtered', cmap="viridis")
    plot_heat_map_from_data(subtracted, Width, Height, args = (fig, ax3), title= 'subtracted', cmap="viridis")
    
    
### file selection and operation widgets  
files      = cwd_files_search_with('.h5')
dropdown   = ipywidgets.Dropdown(options= files, value = files[0], description='File : ', disabled=False)
search_q   = ipywidgets.FloatRangeSlider( value=(1, 2), min=qgrid2[0],   max=qgrid2[-1], step=0.1,   description='q Range:',  disabled=False,  continuous_update=False,  orientation='horizontal',  readout=True, readout_format='.2f',)
n_point    = ipywidgets.IntSlider(value=4,                   min=1,         max=10, step=1, description='N-point : ', continuous_update=False, layout=ipywidgets.Layout(width='30%'))

#### button press event
button     = ipywidgets.Button(description='Go!');                                      button.style.button_color = "green";
button.on_click(on_button_clicked)

display(ipywidgets.HBox([dropdown, search_q, n_point, button ]))
display(out)

Output()

In [14]:
### find heatmap using (I_p - I_avg) -- MultiThreating Processing
%matplotlib widget
import concurrent.futures
from matplotlib import colors

start = time.perf_counter()

#### specs - plaques search
file = '2109_EC-roi1_masked.h5' # '2048_B8_masked.h5' # '2109_EC-roi0_masked.h5' # '1892_EC-roi0_masked.h5' # '2512_EC-roi1_masked.h5' # '2029_Dentate_gyrus-roi0_masked.h5'    # '2029_EC-roi0.h5'    '2029_Dentate_gyrus-roi0_masked.h5'  '2048_B8_masked.h5'
bkg_frame                  = [3469] # [7045,7046] # 973        [3684, 3685]      # background frame

window_size    = 4     # 4 point averaging
normalize_at_q = 1.37  # normalizing at this q
peak_q         = 1.37
# Ipeak at q
first_avg_q    = 1.15  # left q from Ipeak to be averaged
second_avg_q   = 1.47  # right q from Ipeak to be averaged
n_calc_frames  = 800  # get the highest 500 frames
opening_kernel = 2     # 2x2 kernel for morphological operation
n_tissue_frms  = 3     # kernal outputs best 3 potential tissue frames
mf_max = 2.5           # mf must be less than this value
mf_min = 1.0           # mf must be greater than this value

#### specs - tissues search
QSearchStart, QSearchEnd   = 1.55, 1.7                      # scaling regions 295,310 or may be 290,370    1.55, 1.8 
seek_mf                    = (-12,12,0.01)                  # mf goes from -8 to +8 by 0.01
kernal_size                = 7;                             # kernal 7 - 48 frames surrounded by plaque frame
area_minQ = 1.0                                             # min area to search q
area_maxQ = 1.45                                            # max area to search q
tissue_frs_find_alg = 'all-frs-one-cluster'     # 'one-fr-one-cluster'   'all-frs-one-cluster' [2067, 2068]

### make a color map of fixed colors
# cmap = 'tab10'  # discrete_cmap(N=3, base_cmap = 'tab10')
cmap = colors.ListedColormap(['darkblue', 'green' ,'red'])
bounds=[0, 126, 250, 255]
norm = colors.BoundaryNorm(bounds, cmap.N)

### computations
### Ipeak - (Iq1+Iq2)/2 calcuations for potential frames
data = Data_Analysis(file, qgrid2, window_size=4, normalize=False, normalize_at_q=normalize_at_q ,directory=os.getcwd())
Iq   = data.bkg_sub(bkg_frame = bkg_frame)

npatterns = Iq.shape[0]
result = {}; result['diff'] = np.zeros(npatterns) # result['diff'].shape - (7812,)
for frame in range(npatterns):
    result['diff'][frame] = Iq[frame, qgrid_to_indices(qgrid2, peak_q)] - (Iq[frame, qgrid_to_indices(qgrid2, first_avg_q)]+Iq[frame, qgrid_to_indices(qgrid2, second_avg_q)])/2
potential_plaques = np.argsort(result['diff'],axis= None, kind='stable')[::-1][:n_calc_frames]

### morpholocal operations
Width, Height = width_height(file)
sna = snaking(Width, Height)
A_MAT = np.array([np.zeros((Height,Width)),sna])   # zero values matrix (A[0]=0) with frame numbers depth (A[1]=frames)
A = from_clusterFr_ceffs_to_matrix(A_MAT, potential_plaques, np.ones(len(potential_plaques)))
fig, [ax1, ax2, ax3] = plt.subplots(nrows=1, ncols=3, figsize=(17,6))
plot_heat_map_from_data(A[0], Width, Height, args = (fig, ax1), title= f'original - frames-{n_calc_frames}', cmap=cmap)
mask,_ = global_thresholding(A[0], thr=1, binary_inv = False)
_, mask = cv2.threshold(A[0], 0, 255, cv2.THRESH_BINARY)
kernal = np.ones((opening_kernel,opening_kernel), np.uint8)
opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernal)

### clustering variable declarations
labeled_array, num_features = label(opening, np.ones((3,3)))
print('Total clusters = ', num_features)

#### get clusters --> len(clusters)=175
clusters = []
cluster_label = []
for i in np.arange(1,num_features+1,1):  # zero label is for background, rest labels are for clusters
    _ = sna[labeled_array==i]
    clusters.append(_.tolist())
    cluster_label.append(len(_)*[i])
clusters = flatten(clusters)
cluster_label = flatten(cluster_label)

### pandas setting to ignore warning
pd.options.display.max_columns  = None
pd.options.display.max_rows     = None
pd.options.display.max_colwidth = None
pd.options.mode.chained_assignment = None  # default='warn'

### declaring variable
df = pd.DataFrame(columns=['plaque-frame', 'cluster-label' ,'selected-plaque', \
                           'tissue-frames', 'tissue-frames-all', 'tissue-frames-mf', 'tissue-frames-area', \
                           'IqBSTF', 'IqBSTS'], index=np.arange(0,len(clusters),1))
df['plaque-frame']  = clusters
df['cluster-label'] = cluster_label

### find the representative frame from each cluster
df['selected-plaque'] = False                           # initilizing to False
for i in np.arange(1, num_features+1,1):
    cluster_frames = df['plaque-frame'][df['cluster-label']==i].values   # frames in each cluster
    cluster_max_frame = cluster_frames[np.argmax(result['diff'][cluster_frames])]
    df.loc[df.index[df['plaque-frame'] == cluster_max_frame],'selected-plaque'] = True # True only if the frame is the maximum in a cluster
# print(df.loc[df.index[df['selected-plaque']==True], 'plaque-frame'])

### kernal sliding and getting background frames
fr_idx = Snaking_frames_search(Width, Height)   # kernal frame search class

# def kernal_mp(idx, frame):                                  #### uncomment to parallelize
# frames_to_look_for_tissue = df.loc[df.index[df['selected-plaque'] == True], "plaque-frame"].values;  ### peak frames in clusters only - 30 frames for 30 clusters
frames_to_look_for_tissue = df["plaque-frame"].values                                       ### all frames in a cluster
for idx, frame in enumerate(frames_to_look_for_tissue):       #### comment out to parallelize

    if tissue_frs_find_alg == 'one-fr-one-cluster' and df['selected-plaque'][idx] == False:
        continue;
    elif tissue_frs_find_alg == 'all-frs-one-cluster':
        pass;

    ### kernal sliding and getting tissue frames
    tissue_frames = np.array(fr_idx.frame_idx_to_kernal_frames(kernal_size, frame))  # kernal tissue frames

    ### declare variable
    mf   = np.full(npatterns,np.nan);   # do not trust mf - nearby mf (two cluster same frame) is overwritten
    area = np.full(npatterns,np.nan);   # do not trust area - nearby area (two cluster same frame) is overwritten

    ### get tissues
    area_temp = np.zeros(len(tissue_frames))
    for i,tissue_frame in enumerate(tissue_frames):
        mf[tissue_frame], area_temp[i],_,_ = data.tissue_sub(frame, tissue_frame, scale_method = 'MSE', return_alg = 'one_tissue-fr' , \
                                                            area_minQ= area_minQ, area_maxQ = area_maxQ, mf_Qindices = (QSearchStart, QSearchEnd), \
                                                            seek_mf = seek_mf, mf_min=mf_min, mf_max=mf_max , \
                                                            window_size=None, show_result=False);
        area[tissue_frame] = area_temp[i]

    tissue_frames_sorted = tissue_frames[np.argsort(area_temp, axis= None, kind='stable')[::-1] ] # descending order
    tissue_pot = []
    for pot_tissue in tissue_frames_sorted :
        if mf[pot_tissue] <mf_max and mf[pot_tissue] > mf_min :
            tissue_pot.append(pot_tissue)
        if len(tissue_pot)==n_tissue_frms:
            break
    if len(tissue_pot) == 0:
        print('no tissue found for frame - ',frame, ' ; thus dropping this frame ->', np.array(tissue_pot))
        df.drop(index=idx, inplace=True)
        continue;
    df['tissue-frames-all'][idx]  = tissue_frames_sorted
    df['tissue-frames'][idx]      = np.array(tissue_pot)  
    df['tissue-frames-mf'][idx]   = [mf[tissue_frame]   for tissue_frame in df['tissue-frames'][idx]]
    df['tissue-frames-area'][idx] = [area[tissue_frame] for tissue_frame in df['tissue-frames'][idx]]
    
    df['IqBSTF'][idx], df['IqBSTS'][idx] = data.tissue_sub(frame, df['tissue-frames'][idx], scale_method = 'MSE', return_alg = 'multi_tissue-fr' , \
                                                    area_minQ= area_minQ, area_maxQ = area_maxQ, mf_Qindices = (QSearchStart, QSearchEnd), \
                                                    seek_mf = seek_mf, mf_min=mf_min, mf_max=mf_max , \
                                                    window_size=window_size, show_result=False);
    df['IqBSTS'][idx] = df['IqBSTS'][idx].flatten()
    df['IqBSTF'][idx] = df['IqBSTF'][idx].flatten()

#### uncomment to parallelize
# with concurrent.futures.ThreadPoolExecutor(max_workers=None) as executor:
#     results = executor.map(kernal_mp, np.arange(len(df["plaque-frame"].values)), df["plaque-frame"].values)

### resetting indices as some frames might have dropped
df.reset_index(drop=True,inplace=True)  # drop parameter to avoid the old index being added as a column

### plot truncated tissues on top of opening images - (clustering results same as opening)
A_MAT = np.array([np.full((Height,Width),0),sna])   # zero values matrix (A[0]=0) with frame numbers depth (A[1]=frames)
B = from_clusterFr_ceffs_to_matrix(A_MAT, cluster=df['plaque-frame'], coeffs=np.full_like(df['plaque-frame'], 255, dtype=int))
plot_heat_map_from_data(B[0], Width, Height, args = (fig, ax2), title= f'clustered total frames-{len(df["plaque-frame"])}', cmap=cmap)

tissues = []
[tissues.append(i.tolist()) for i in df['tissue-frames'].values.tolist() if np.any(np.isnan(i)) == False]  ### for selctive case run only NOT nan frames
tissues = np.unique(flatten(tissues))
A_MAT = np.array([np.zeros((Height,Width)),sna])   # zero values matrix (A[0]=0) with frame numbers depth (A[1]=frames)
C = from_clusterFr_ceffs_to_matrix(A_MAT, cluster= tissues, coeffs=np.full_like(tissues, 127, dtype=int))
plot_heat_map_from_data(np.clip(B[0]+C[0],0,255), Width, Height, args = (fig, ax3), title= f'norm. and peak find at q{peak_q}', cmap=cmap, norm=norm, ticks = [0, 127, 255])

plt.suptitle(file)
plt.tight_layout()

#### Scroll through frames
from sklearn.preprocessing import StandardScaler
start, end = 130, 490                                       # ipywidget plot and classifer

### X array-like of shape (n_samples, n_features)
X = np.zeros((len(df), len(qgrid2)))
for idx in range(len(df)):
    X[idx,:] = df['IqBSTF'][idx]
X = StandardScaler().fit_transform(X)  # Standard Scalar
X = X[:,start:end]                     # X is what you see on plt

### ipywidget plot
plt.figure(num=file)
def plaque_bkg_plots(index):
    plaque = df['plaque-frame'].loc[index]
    plt.cla()
    plt.plot(data.qgrid[start:end], data.Iq[plaque][start:end],          color= 'red',        label = 'Iq' )
    plt.plot(data.qgrid[start:end], data.IqBS[plaque][start:end],        color= 'red',        label = 'IqBS'  )
    plt.plot(data.qgrid[start:end], df['IqBSTF'].loc[index][start:end],  color= 'green',      label = 'IqBSTF')
    plt.plot(data.qgrid[start:end], df['IqBSTS'].loc[index][start:end],  color= 'red',        label = 'IqBSTS')
    plt.legend()
    plt.title(f'plaque-fr-{plaque}, group-{df["cluster-label"][index]}, index:{index}')
    #plt.axis([None,None,None,0.12*10**6])

plt.tight_layout()
    
if tissue_frs_find_alg == 'one-fr-one-cluster' :
    scroll_frames = df.index[df['selected-plaque'] == True]   # df.index[df['selected-plaque'] == True]    # df.index.values
elif tissue_frs_find_alg == 'all-frs-one-cluster' :
    scroll_frames = df.index.values   # df.index[df['selected-plaque'] == True]    # df.index.values

index = ipywidgets.SelectionSlider(value = scroll_frames[0], options = scroll_frames)
ipywidgets.interact(plaque_bkg_plots, index=index)

#### display dataframe
print(df.iloc[:,:4])

### create csv file
print('csv file generated in ',os.getcwd())
df.to_csv('result.csv')

finish = time.perf_counter();    print(f'finished in {(finish-start)//60} minutes and {(finish-start)%60} seconds')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Total clusters =  17


/Users/bashit.a/miniforge3/envs/py38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/bashit.a/miniforge3/envs/py38/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


no tissue found for frame -  1313  ; thus dropping this frame -> []
no tissue found for frame -  561  ; thus dropping this frame -> []
no tissue found for frame -  1307  ; thus dropping this frame -> []
no tissue found for frame -  936  ; thus dropping this frame -> []


/Users/bashit.a/miniforge3/envs/py38/lib/python3.8/site-packages/sklearn/utils/extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/bashit.a/miniforge3/envs/py38/lib/python3.8/site-packages/sklearn/utils/extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(SelectionSlider(description='index', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, …

     plaque-frame  cluster-label  selected-plaque       tissue-frames
0            3906              1             True  [3782, 3903, 3779]
1            3907              1            False  [3782, 3903, 3906]
2            3905              1            False  [3782, 3903, 3906]
3            3904              1            False  [3782, 3903, 3779]
4            3780              1            False  [3782, 3903, 3657]
5            3781              1            False  [3782, 3903, 3657]
6            3933              2            False  [3932, 3877, 3804]
7            3934              2             True  [3932, 3748, 3811]
8            3935              2            False  [3938, 3747, 3748]
9            3936              2            False  [3746, 3872, 3938]
10           3937              2            False  [3746, 3745, 3872]
11           3878              2            False  [3680, 3932, 3678]
12           3877              2            False  [3680, 3932, 3748]
13           3876   

In [84]:
#### plotting individual frames - requires running previous cell
%matplotlib widget
frame = 2068

tissue_frames = np.array(fr_idx.frame_idx_to_kernal_frames(kernal_size, frame))  # kernal tissue frames

### declare variable
mf   = np.full(npatterns,np.nan);   # do not trust mf - nearby mf (two cluster same frame) is overwritten
area = np.full(npatterns,np.nan);   # do not trust area - nearby area (two cluster same frame) is overwritten

### get tissues
area_temp = np.zeros(len(tissue_frames))
for i,tissue_frame in enumerate(tissue_frames):
    mf[tissue_frame], area_temp[i],_,_ = data.tissue_sub(frame, tissue_frame, scale_method = 'MSE', return_alg = 'one_tissue-fr' , \
                                                        area_minQ= area_minQ, area_maxQ = area_maxQ, mf_Qindices = (QSearchStart, QSearchEnd), \
                                                        seek_mf = seek_mf, mf_min=mf_min, mf_max=mf_max , \
                                                        window_size=None, show_result=False);
    area[tissue_frame] = area_temp[i]

tissue_frames_sorted = tissue_frames[np.argsort(area_temp, axis= None, kind='stable')[::-1] ] # descending order
tissue_pot = []
for pot_tissue in tissue_frames_sorted :
    if mf[pot_tissue] <mf_max and mf[pot_tissue] > mf_min :
        tissue_pot.append(pot_tissue)
    if len(tissue_pot)==n_tissue_frms:
        break

print(np.array(tissue_pot))

IqBSTF, IqBSTS = data.tissue_sub(frame, tissue_frames_sorted, scale_method = 'MSE', return_alg = 'multi_tissue-fr' , \
                                                area_minQ= area_minQ, area_maxQ = area_maxQ, mf_Qindices = (QSearchStart, QSearchEnd), \
                                                seek_mf = seek_mf, mf_min=mf_min, mf_max=mf_max , \
                                                window_size=window_size, show_result=False);

plt.plot(data.qgrid[start:end], data.Iq[frame][start:end],          color= 'red',        label = 'Iq'    )
plt.plot(data.qgrid[start:end], data.IqBS[frame][start:end],        color= 'red',     label = 'IqBS'  )
plt.plot(data.qgrid[start:end], IqBSTF[0][start:end],  color= 'green',      label = 'IqBSTF')
plt.plot(data.qgrid[start:end], IqBSTS[0][start:end],  color= 'red',      label = 'IqBSTS')


/Users/bashit.a/miniforge3/envs/py38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/bashit.a/miniforge3/envs/py38/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[2193 2065 1960]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
### requires to run previous cell
X = Data_Analysis('2048_B8_masked.h5', qgrid2, window_size=4, normalize=False, normalize_at_q=normalize_at_q ,directory='/Users/bashit.a/Documents/Alzheimer/Dec-2020/')
_   = X.bkg_sub(bkg_frame = [2222])
plaque = X.IqBS[2068]

tissue = IqBSTF[0]

X = Data_Analysis('2109_EC-roi0.h5', qgrid2, window_size=4, normalize=False, normalize_at_q=normalize_at_q ,directory='/Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sample-7/')
_   = X.bkg_sub(bkg_frame = [500])
tau = X.IqBS[5794]

In [122]:
# requires to run previous cell
%matplotlib widget
print(plaque.shape)
plt.plot(data.qgrid[start:end], plaque[start:end],          color= 'red',        label = 'plaque'    )
plt.plot(data.qgrid[start:end], tissue[start:end],        color= 'purple',     label = 'tissue'  )
plt.plot(data.qgrid[start:end], tau[start:end],  color= 'green',      label = 'tau')
plt.legend()

(690,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
### Image registration map creation - independent cell
from matplotlib import colors

%matplotlib widget
frames = [374, 1040, 983, 3874, 2744, 2187, 1865]      # where center of the kernal
Width = 63                # image width
Height = 63               # image height
kernal_size = 5;                             # kernal 7 - 48 frames surrounded by plaque frame
color_frames = [1165]     # manually color any frames
cmap = colors.ListedColormap(['darkblue', 'green', 'red'])

### kernal sliding and getting background frames
fr_idx = Snaking_frames_search(Width, Height)                                    # kernal frame search class
tissue_frames = np.array([])
sna = snaking(Width, Height)

for frame in frames:
    tissue_frames_temp = fr_idx.frame_idx_to_kernal_frames(kernal_size, frame)
    print(tissue_frames_temp)
    tissue_frames = np.append(tissue_frames,tissue_frames_temp)                  # kernal tissue frames

tissue_frames = tissue_frames.flatten()
A_MAT = np.array([np.zeros((Height,Width)),sna])   # zero values matrix (A[0]=0) with frame numbers depth (A[1]=frames)
A = from_clusterFr_ceffs_to_matrix(A_MAT, tissue_frames, np.full(len(tissue_frames),0.5))

B = from_clusterFr_ceffs_to_matrix(A, color_frames, np.full(len(tissue_frames),1))

fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(17,6))
plot_heat_map_from_data(B[0], Width, Height, args = (fig, ax1), title= f'original - frames-', cmap=cmap)

[250, 249, 248, 247, 246, 253, 254, 255, 256, 257, 376, 375, 374, 373, 372, 379, 380, 381, 382, 383, 502, 501, 500, 499, 498]
[912, 913, 914, 915, 916, 977, 976, 975, 974, 973, 1038, 1039, 1040, 1041, 1042, 1103, 1102, 1101, 1100, 1099, 1164, 1165, 1166, 1167, 1168]
[859, 858, 857, 856, 855, 904, 905, 906, 907, 908, 985, 984, 983, 982, 981, 1030, 1031, 1032, 1033, 1034, 1111, 1110, 1109, 1108, 1107]
[3750, 3749, 3748, 3747, 3746, 3809, 3810, 3811, 3812, 3813, 3876, 3875, 3874, 3873, 3872, 3935, 3936, 3937, 3938, 3939]
[2620, 2619, 2618, 2617, 2616, 2671, 2672, 2673, 2674, 2675, 2746, 2745, 2744, 2743, 2742, 2797, 2798, 2799, 2800, 2801, 2872, 2871, 2870, 2869, 2868]
[2059, 2060, 2061, 2062, 2063, 2098, 2097, 2096, 2095, 2094, 2185, 2186, 2187, 2188, 2189, 2224, 2223, 2222, 2221, 2220, 2311, 2312, 2313, 2314, 2315]
[1741, 1740, 1739, 1738, 1737, 1786, 1787, 1788, 1789, 1790, 1867, 1866, 1865, 1864, 1863, 1912, 1913, 1914, 1915, 1916, 1993, 1992, 1991, 1990, 1989]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
# comparing two duplicate dataframe, result is empty table
df1 = pd.read_csv('result.csv')
df2 = pd.read_csv('result-now.csv')
pd.concat([df1,df2]).drop_duplicates(keep=False)

In [109]:
cluster = 14
cluster_frames = df['plaque-frame'][df['cluster-label']==cluster].values   # frames in each cluster
print(list(cluster_frames))

[5270, 5271, 5272, 5187, 5186, 5185, 5144, 5145, 5146]


In [170]:
#### 1d plot for peak frames in each cluster
%matplotlib widget

peak_fr_in_cluster = df.iloc[df.index[df['selected-plaque']==True]]['plaque-frame'].values
print(peak_fr_in_cluster)
# print(peak_fr_in_cluster )
sorted_peak_fr_in_cluster = peak_fr_in_cluster[np.argsort(np.nanmax(Iq[peak_fr_in_cluster,start:end], axis=1))[::-1]]  #hightest to lowest

plt.figure()
def plot(frame):
    plt.cla()
    plt.plot(qgrid2[start:end], Iq[frame,start:end])

slider = ipywidgets.SelectionSlider(value=sorted_peak_fr_in_cluster[0], options=sorted_peak_fr_in_cluster)
ipywidgets.interact(plot, frame = slider)
print(qgrid2[start], qgrid2[end])

[7388 7373 6895 6068 5422 5271 4517 3993 3275 2177 1710 1589 1097  832
  734  420   45]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(SelectionSlider(description='frame', options=(6068, 6895, 7388, 5422, 1710, 2177, 4517, …

0.40000000000000024 2.200000000000002


In [14]:
%matplotlib widget
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation


# https://stackoverflow.com/questions/39767612/what-is-the-equivalent-of-matlabs-imadjust-in-python
def imadjust(x,a,b,c,d,gamma=1):
    # Similar to imadjust in MATLAB.
    # Converts an image range from [a,b] to [c,d].
    # The Equation of a line can be used for this transformation:
    #   y=((d-c)/(b-a))*(x-a)+c
    # However, it is better to use a more generalized equation:
    #   y=((x-a)/(b-a))^gamma*(d-c)+c
    # If gamma is equal to 1, then the line equation is used.
    # When gamma is not equal to 1, then the transformation is not linear.

    y = (((x - a) / (b - a)) ** gamma) * (d - c) + c
    return y

In [16]:
### KMeans model
def train_KMeans(X,n_clusters):
    kmeans = KMeans(init="k-means++", n_clusters=n_clusters, n_init=4)
    kmeans.fit(X)
    labels = kmeans.predict(X)
    return labels
labels = train_KMeans(X, n_clusters=2)
df['KMeans'] = labels

### GMM model
def train_GMM(X, n_components):
    from sklearn.mixture import GaussianMixture               # import package
    gaussian_model = GaussianMixture(n_components=n_components)          # define the model
    gaussian_model.fit(X)                                     # train the model
    labels = gaussian_model.predict(X)                        # assign each data point to a cluster
    return labels

# labels = train_GMM(X, n_components=2)
# df['GMM'] = labels

### Affinity Model
def train_Affinity(X):
    af = AffinityPropagation(preference=-20000, random_state=0).fit(X)
    cluster_centers_indices = af.cluster_centers_indices_
    print('number of clusters - ', len(cluster_centers_indices))
    labels = af.labels_
    return labels

# labels = train_Affinity(X)
# df = df.assign(Affinity=labels)

print('csv file generated in ',os.getcwd())
df.to_csv('result.csv')

csv file generated in  /Users/bashit.a/Documents/Alzheimer/July-2021/BNL-Data/sample-6


In [19]:
%matplotlib widget

fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
plot_heat_map_from_data(B[0], Width, Height, args = (fig, ax1), title= f'clustered - frames-{len(df["plaque-frame"])}', cmap=cmap)

A_MAT = np.array([np.zeros((Height,Width)),sna])   # zero values matrix (A[0]=0) with frame numbers depth (A[1]=frames)
D = from_clusterFr_ceffs_to_matrix(A_MAT, cluster=df['plaque-frame'].values.astype(np.uint32), coeffs=np.uint8(imadjust(labels,labels.min(),labels.max(),0,255)))
plot_heat_map_from_data(D[0], Width, Height, args = (fig, ax2), title= f'classifier categories-{len(np.unique(labels))}', cmap=cmap)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
sna

array([[7749, 7750, 7751, ..., 7809, 7810, 7811],
       [7748, 7747, 7746, ..., 7688, 7687, 7686],
       [7623, 7624, 7625, ..., 7683, 7684, 7685],
       ...,
       [ 188,  187,  186, ...,  128,  127,  126],
       [  63,   64,   65, ...,  123,  124,  125],
       [  62,   61,   60, ...,    2,    1,    0]])

In [49]:
### 2-d plot animation
%matplotlib widget

from matplotlib.animation import FuncAnimation
from IPython import display

x_data = []
y_data = []

# Plot setups
fig, ax = plt.subplots()
plt.axis('scaled')   # making both x and y axis scaled
ax.set_xlim(0,20)
ax.set_ylim(0,200)
line, = ax.plot([])

# things that are going to change in every frame
def animate(frame):
    
    # append new data/update plot 
    x_data.append(frame)
    y_data.append(frame*10)
    
    # plot new data
#     line.set_xdata(x_data)
#     line.set_ydata(y_data)
    line.set_data((x_data,y_data))

    return line, # return plotting object for each frame call

anim = FuncAnimation(fig, func=animate, frames=np.arange(0,10,0.1), interval=100)   # smooth --> interval = 20, frames =100 big changes --> interval = 100
video = anim.to_html5_video()   # visualize in jupyter-lab
html = display.HTML(video)      # convert to html
display.display(html)           # display HTML file
plt.close()                     # close plot

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
### 3-d plot animation

%matplotlib widget
X = np.array([[1,1,1], [2,2,3],[10,9,8]])
labels = np.array([1,2,10])
plot_3d_animation(X, labels,)

In [17]:
### FTIR data import
%matplotlib widget

mat_fname = '/Users/bashit.a/Documents/Alzheimer/FTIR/300x300Test.mat'
Width, Height = 48, 48

window_size = 4  # 4 point average
f = h5py.File(mat_fname, 'r')
wn = np.array(f['vicD']['wn'])
wn = wn[0]
# filtering
window      = np.ones(window_size)/window_size                                    # window_size = 4 --> 0.25,0.25,0.25,0.225
ftir = np.array(f['vicD']['r'])
ftir = np.transpose(np.array([np.convolve(window, ftir[:,idx], mode='same') for idx in range(ftir.shape[1])  ])) if window_size>1 else ftir   # filter output using convolution

f, axs = plt.subplots(nrows=4, ncols=3, figsize = (16,8))
frame_cor = np.arange(0,Width*Height).reshape(Width, Height)                  # indices for heat map

def ftir_heatmap(wave_number, frame, tissue, mica):
    
    ########## --------- matplotlib mouse hovering function for snaking --------- ##########
    numrows, numcols = Height, Width                  # format_coord function requires this global variables
    def format_coord(x, y):
        col = int(x)                                  # truncate x values
        row = int(y)                                  # truncate y values
        if 0 <= col < numcols and 0 <= row < numrows:
            z = frame_cor[row, col]        # flipping to get correct value of z     
            return 'x=%1.2f, y=%1.2f, FRAME=%d' % (x, y, z)
        else:
            return 'x=%1.2f, y=%1.2f' % (x, y)        # outside the plotting range, no need 
    
    axs[0,:2] = plt.subplot(4, 2, 1)
    axs[0,0].imshow(ftir[wn==wave_number,:].reshape(Width, Height), cmap='jet')
    axs[0,0].format_coord = format_coord
    axs[0,0].set(title='Heatmap FTIR')

    stds = np.zeros(len(wn))
    for wn_ in range(len(wn)):
        stds[wn_] = np.std(ftir[wn_,:])
    
    axs[0,2].clear()
    axs[0,2].plot(wn, stds,'blue')
    axs[0,2].scatter(wn[wn==wave_number], stds[wn==wave_number],color='red', marker = 'o')
    axs[0,2].set(xlabel = "wn", ylabel = "std")
    
    axs[1,0].cla()
    axs[1,0].plot(wn,ftir[:, frame], label=f'fr-{frame}')
    axs[1,0].set(xlabel = "wn", ylabel = "absorbtion")
    axs[1,0].legend()
    
    axs[1,1].cla()
    axs[1,1].plot(wn,ftir[:, tissue], label=f'tissue-{tissue}')
    axs[1,1].set(xlabel = "wn", ylabel = "absorbtion")
    axs[1,1].legend()
    
    axs[1,2].cla()
    axs[1,2].plot(wn,ftir[:, mica], label=f'mica-{mica}')
    axs[1,2].set(xlabel = "wn", ylabel = "absorbtion")
    axs[1,2].legend()
    
    axs[2,:] = plt.subplot(4, 1, 3)
    axs[2,0].cla()
    axs[2,0].plot(wn,ftir[:, frame] -ftir[:, mica], label=f'fr-{frame}-mica-{mica}')
    axs[2,0].plot(wn,ftir[:, tissue]-ftir[:, mica], label=f'tissue-{tissue}-mica-{mica}')
    axs[2,0].set(xlabel = "wn", ylabel = "absorbtion")
    axs[2,0].legend()
    
    axs[3,:] = plt.subplot(4, 1, 4)
    axs[3,0].cla()
    axs[3,0].plot(wn,(ftir[:, frame] -ftir[:, mica]) - (ftir[:, tissue]-ftir[:, mica]), label=f'fr-{frame}-tissue-{tissue}-2*mica-{mica}')
    axs[3,0].set(xlabel = "wn", ylabel = "absorbtion")
    axs[3,0].legend()

plt.tight_layout()
frame = ipywidgets.IntSlider(min=frame_cor.min(), max=frame_cor.max(), value = 670, continuous=False)   # value = int(frame_cor.mean())
tissue = ipywidgets.IntSlider(min=frame_cor.min(), max=frame_cor.max(), value = 899, continuous=False)
mica = ipywidgets.IntSlider(min=frame_cor.min(), max=frame_cor.max(), value = 521, continuous=False)

wave_number = ipywidgets.IntSlider(min=np.min(wn), max=np.max(wn), value = 1625, continuous=False)
ipywidgets.interact(ftir_heatmap, wave_number=wave_number, frame=frame, tissue=tissue, mica=mica)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1625, description='wave_number', max=4000, min=750), IntSlider(value=670…

<function __main__.ftir_heatmap(wave_number, frame, tissue, mica)>